In [1]:
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
cnx = create_engine('postgresql://ubuntu@34.214.216.222:5432/ponv')

In [3]:
# Query to get surgery procedures translated and a count
query = '''
select pt.procedure_en, count(1)
from ponvfull pf
INNER JOIN proc_trans pt ON pt.surgical_procedure = pf.surgical_procedure 
group by pt.surgical_procedure, pf.surgical_procedure 
order by count(1) DESC
'''

proc_table_en = pd.read_sql_query(query, cnx)
proc_table_en.head(10)

OperationalError: (psycopg2.OperationalError) could not connect to server: Operation timed out
	Is the server running on host "34.214.216.222" and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
# Script to create statements to dummify surgeries
for proc in proc_table_en.iloc[:,0]:
    def_string = 'count(CASE WHEN procedure_en = '
    def_string2 = 'THEN 1 END) AS '
    print(def_string+"'"+proc+"'"+def_string2+proc.replace(' ','_')+',')

In [ ]:
proc_dummy_query = '''
SELECT  trtbl.patientid, trtbl.procedure_en,
count(CASE WHEN procedure_en = 'Gastrointestinal'THEN 1 END) AS Gastrointestinal,
	count(CASE WHEN procedure_en = 'Mastectomy'THEN 1 END) AS Mastectomy,
	count(CASE WHEN procedure_en = 'Thoracic'THEN 1 END) AS Thoracic,
	count(CASE WHEN procedure_en = 'Nephrectomy'THEN 1 END) AS Nephrectomy,
	count(CASE WHEN procedure_en = 'Hysterectomy'THEN 1 END) AS Hysterectomy,
	count(CASE WHEN procedure_en = 'Exploratory Laparotomy'THEN 1 END) AS Exploratory_Laparotomy,
	count(CASE WHEN procedure_en = 'Other'THEN 1 END) AS Other,
	count(CASE WHEN procedure_en = 'Spine Surgery'THEN 1 END) AS Spine_Surgery,
	count(CASE WHEN procedure_en = 'Cystectomy'THEN 1 END) AS Cystectomy,
	count(CASE WHEN procedure_en = 'Prostatectomy'THEN 1 END) AS Prostatectomy,
	count(CASE WHEN procedure_en = 'Hepatectomy'THEN 1 END) AS Hepatectomy,
	count(CASE WHEN procedure_en = 'Plastic'THEN 1 END) AS Plastic,
	count(CASE WHEN procedure_en = 'Cytoreduction'THEN 1 END) AS Cytoreduction,
	count(CASE WHEN procedure_en = 'Anexectomy/Ovariectomy/'THEN 1 END) AS Anexectomy_Ovariectomy,
	count(CASE WHEN procedure_en = 'Head/Neck'THEN 1 END) AS Head_Neck,
	count(CASE WHEN procedure_en = 'Orthopedic'THEN 1 END) AS Orthopedic,
	count(CASE WHEN procedure_en = 'Hysterectomy VLP'THEN 1 END) AS Hysterectomy_VLP,
	count(CASE WHEN procedure_en = 'Extensive Lymphadenectomy'THEN 1 END) AS Extensive_Lymphadenectomy,
	count(CASE WHEN procedure_en = 'Esophagectomy'THEN 1 END) AS Esophagectomy,
	count(CASE WHEN procedure_en = 'Breast Lumpectomy'THEN 1 END) AS Breast_Lumpectomy,
	count(CASE WHEN procedure_en = 'Pancreatectomy'THEN 1 END) AS Pancreatectomy,
	count(CASE WHEN procedure_en = 'Hip Arthoplasty'THEN 1 END) AS Hip_Arthoplasty,
	count(CASE WHEN procedure_en = 'Soft Tissue Resection'THEN 1 END) AS Soft_Tissue_Resection,
	count(CASE WHEN procedure_en = 'Limb Amputation'THEN 1 END) AS Limb_Amputation,
	count(CASE WHEN procedure_en = 'Gallbladder'THEN 1 END) AS Gallbladder
FROM 
(SELECT patientid, pt.surgical_procedure, pt.procedure_en
FROM ponvfull pf
INNER JOIN proc_trans pt ON pf.surgical_procedure = pt.surgical_procedure) AS trtbl
GROUP BY trtbl.patientid, trtbl.procedure_en;
'''

In [ ]:
# Select primary data set with populated PONV data
sample_select_query = '''
SELECT *
FROM analysis_set
''' 
raw_df = pd.read_sql_query(sample_select_query, cnx)

In [ ]:
raw_df.info()

Addressing Nulls
1) ignore chemo emetogenicity
2) need to go back and figure out what's wrong with the neuraxial opioid null values


In [ ]:
'''(no_dropnull['anesthesia_technique'] == 4)&
no_dropnull = raw_df[raw_df['neuraxial_opioid'].notnull()]
no_dropnull['neuraxial_opioid'].map({True:1, False:0})
no_dropnull[(no_dropnull['neuraxial_opioid'] == ' ')].iloc[:,15:17]
no_dropnull[(no_dropnull['neuraxial_opioid'] == None)]
''''''

## Drop reasons:
1) nausea_24h: somewhat a symptom of outcome and therefore too correlated with ponv\
2) sex: gender_code replaces\
3) chemotherapy_emetogenicity: duplicative given other features and too many NaN\
4) neuraxial_opioid: unable to identify NaN values; will readdress if needed

In [ ]:
analysis_x = raw_df.drop(['ponv','nausea_24h', 'sex', 'chemotherapy_emetogenicity', 'neuraxial_opioid'], axis=1)
analysis_y = raw_df['ponv']

In [ ]:
analysis_x = analysis_x[analysis_x['fentanil_mcg'] < 3000]

In [ ]:
#set nan ketamine dose to median
analysis_x.loc[308,'ketamine_dose'] = analysis_x[analysis_x['ketamine_dose'] > 0]['ketamine_dose'].median()

In [ ]:
analysis_x.loc[308,'ketamine_dose']

In [ ]:
#set nan ketamine dose to median
analysis_x['intraoperative_morphine'].fillna(False, inplace=True)

In [ ]:
analysis_x.info()

In [ ]:
# Filling in morphine dose
median_morph = analysis_x[analysis_x['intraoperative_morphine_dose']>0]['intraoperative_morphine_dose'].median()
analysis_x['intraoperative_morphine_dose']=median_morph

In [ ]:
tv_x, test_x, tv_y, test_y = train_test_split(analysis_x, analysis_y, test_size=.2, random_state=10)
train_x, val_x, train_y, val_y = train_test_split(tv_x, tv_y, test_size=.25, random_state=444)

In [ ]:
train_x.columns

In [ ]:
apfel_feat = ['gender_code', 'non_smoker', 'previous_ponv', 'postoperative_opioids']
pp_data = train_x[apfel_feat]
pp_data['ponv'] = train_y

In [ ]:
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

sns.barplot(pp_data['gender_code'], pp_data['ponv'], ax=ax1)

sns.barplot(pp_data['non_smoker'], pp_data['ponv'], ax=ax2)

sns.barplot(pp_data['previous_ponv'], pp_data['ponv'], ax=ax3)

sns.barplot(pp_data['postoperative_opioids'], pp_data['ponv'], ax=ax4)

## Anesthesia technique


In [ ]:
train_x['anesthesia_technique'].value_counts()

In [ ]:
fig = plt.figure(figsize=(16,6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
sns.barplot(train_x['anesthesia_technique'], train_y, ax=ax1)
sns.kdeplot(train_x['anesthesia_technique'], ax=ax2)

Although a key wasn't available to explain what each of the techniques were, I don't believe further analysis is required given the ponv rate for all the techniques seem pretty similar. *I may need to do a hypothesis test to justify?*

## Baseline Random Forest Regressor vs. Apfel

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
baseline_val_x = val_x[apfel_feat]

In [ ]:
# Baseline RFC model
baseline_x = pp_data.drop('ponv', axis=1)
baseline_y = pp_data['ponv']
baseline_rfc = RandomForestClassifier()
baseline_rfc.fit(baseline_x, baseline_y)
rfc_pred = baseline_rfc.predict(baseline_val_x)
accuracy_score(val_y, rfc_pred)

In [ ]:
sum(tv_y)/len(tv_y)

In [ ]:
baseline_rfc.predict_proba(baseline_val_x)[:10,0]

In [ ]:
baseline_comp = val_x[['patientid', 'apfel']]

In [ ]:
baseline_comp['apfel_prob'] = baseline_comp['apfel'].map({0:.1, 1:.2, 2:.4, 3:.6, 4:.8})

In [ ]:
baseline_comp.shape

In [ ]:
baseline_comp['baseline_rfc'] = baseline_rfc.predict_proba(baseline_val_x)[:,0]

In [ ]:
baseline_comp['actual'] = val_y

In [ ]:
baseline_comp.drop('actual', axis=1, inplace=True)

In [ ]:
baseline_comp['apfel_pred'] = baseline_comp['apfel_prob'].apply(lambda x: True if x > 0.5 else False)

In [ ]:
baseline_comp['rfc_pred'] = baseline_rfc.predict(baseline_val_x)

In [ ]:
baseline_comp['actual'] = val_y

In [ ]:
# Summary of apfel test vs. random forest
baseline_comp

In [ ]:
fig = plt.figure()
ax = sns.heatmap(confusion_matrix(baseline_comp['actual'], baseline_comp['apfel_pred']), cmap='Greens', 
            annot=True, fmt='0.2f', robust = True, yticklabels=['False', 'True'],
            xticklabels=['False', 'True'])
plt.title('Apfel Scale Confusion Matrix', fontsize=20)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
#plt.savefig('../ppt/Apfel_cm.png')

In [ ]:
sns.heatmap(confusion_matrix(baseline_comp['actual'], baseline_comp['rfc_pred']), cmap='Blues', 
            annot=True, fmt='0.2f', linecolor='black', robust=True, yticklabels=['False', 'True'],
            xticklabels=['False', 'True'])
plt.title('Random Forest Confusion Matrix', fontsize=20)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
#plt.savefig('../ppt/RFC_cm.png')

# Resampling

In [ ]:
import imblearn.over_sampling

In [ ]:
n_pos = np.sum(train_y == 1)
n_neg = np.sum(train_y == 0)
ratio = {1:n_pos * 2, 0 : n_neg}

ROS = imblearn.over_sampling.RandomOverSampler(sampling_strategy = ratio, random_state=444)

train_x_os, train_y_os = ROS.fit(train_x, train_y)


# Feature Engineering

## Anesthetic effect on PONV

In [ ]:
opioid_feat = train_x.iloc[:,15:27]
opioid_feat['ponv'] = train_y.map({True:1, False:0})

In [ ]:
opioid_feat.info()

In [ ]:
neurax_test = opioid_feat.iloc[:,0]
neurax_test = pd.concat([neurax_test,train_y], axis=1)
neurax_test.dropna(inplace=True)
neurax_test.groupby('neuraxial_opioid').agg({'ponv':'mean'})

In [ ]:
opioid_feat

In [ ]:
opi_bool = ['remifentanil', 'continuous_sufentanil', 'tramadol_pacu', 'intraoperative_ketamine', 'intraoperative_morphine']

for drugs in opi_bool:
    print(opioid_feat.groupby(drugs).agg({'ponv':'mean'}))

In [ ]:
'''additional follow up on effects of continous sufentanil and tramadol; might be a good exercise to do hypothesis
testing to prove difference is significant but probably fine to eyeball it?'''
from scipy.stats import ttest_rel

## Chemo effect on PONV

## PONV for different surgeries

<span style="color:red"> <b> Need to get rid of the fentanyl outlier </b> (3,000 mcg) </span>

In [ ]:
plt.scatter(anes_feat.index, anes_feat['fentanil_mcg'])